In [3]:
import face_recognition
import redis
import pandas as pd
import numpy as np
import cv2

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [4]:
#connection to database
#Connect to redis client

hostname='redis-19868.c267.us-east-1-4.ec2.cloud.redislabs.com'
portnumber=19868
password='ElZFvkWqGrcayZt7SMmnmknmc9rX5nUC'

r=redis.StrictRedis(host=hostname,
                    port=portnumber,
                    password=password)

In [5]:
r.ping()

True

In [6]:
name='class:register'
retrive_dict=r.hgetall(name)

In [8]:
retrive_dict

{b'Mallavarapu Sai@1044': b'n\xc3)\xbf\x9f$\x1f?\xf1\xa2v\xbf]\xe8-?h\x0b=\xbfxj\xf9?\x91\xe3\x12>\xd3\x80\xeb?a\xea\x8a\xbeC\xcb\xe7=\r\xcf\xe0>]^\x8e?\x18`*\xbf-\xe0\x88\xbf\xcc\n]\xbe\xadi\x8c\xbfo\x8b\xda\xbe\x9b\xc9_>\xef\x90\x86\xbf\xd15\t?0\xcb\xfd>Q!\x95?QaU\xbf\'\x88U>\xbb\x0f\x86=\xf0\x91\xc7\xbfU\xd4[\xbf\x10K\xb1>\xbf\x83\x1e?\xe3\xc8\xbe\xbe\xa81\xbe\xbe\x90\xf4\x83>eG\xd0\xbec\x89\xef\xbd%\x8b\xe5\xbc\x0b\x08\xde?\x91B\xa5\xbf\x9b*\xb0?\t\x92\x91\xbeU\x04\xb4?\x93\x88\xbe\xbe\xa5XN\xbf\xf0\xe2\x00>\xc0\xe4\x18>0"\xe4=\xf7\x0c&?5\x83y?\x8d\x18=\xbe\x1d=\x83\xbfM\nI\xbe\xef-\xa0?\xc8\x87\x91?w\xa5J\xbe\xe5\x81L\xbd\xd7\x96\xb1\xbf#\x8b\x84\xbf&\x05\xa4\xbfO\xc7\x01\xbex)(\xbf\'\xf0\xa2?3&\xdb\xbe\xbf\xbad?\x0b\xca\xad\xbf\x1d\xca\xd9?\t\x00\xcd>\xa4b\xd3>I\xd4\xc4>\x01\xec\xc5\xbf\x8d\xac\x06\xbe\xc9\x85\x9b\xbf\xfb\xf3\xe7\xbf\xb7\x84^\xbf\x85:D\xbf{\xa8\xff\xbe\x0f{\x07\xbfsHQ?5\xcc\xa3\xbfC~d\xbec&=?W\x85/?\xb7\xc9\x12?\xf3\x91g>\x0f\xfd\x10@\xf35\x1e\xbf\x05\x0f*\xbf\xf

In [7]:
retrive_series=pd.Series(retrive_dict)
retrive_series=retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index=retrive_series.index
index=list(map(lambda x: x.decode(),index))
retrive_series.index=index
retrive_series

Mallavarapu Sai@1044        [-0.6631383, 0.62165254, -0.9634238, 0.6793268...
Darshil Patel@1024          [0.63586926, 1.2339267, 2.0949066, -0.10146476...
Pradeep Kumar@1058          [1.2769926, 0.40290147, -0.4040916, 1.5141966,...
Kodudula Niketh@1040        [0.41742858, 0.9092604, 1.1397063, -1.0795174,...
Panchangam Akhilesh@1053    [1.3760132, -0.30991262, 0.91287786, 1.0763574...
                                                  ...                        
Aditya Mastud@1005          [-0.4629953, -0.071729705, -0.083244145, -0.42...
S Ruthvik@1064              [0.81026226, -0.01492546, 0.75623745, 1.347576...
Vedant Upadhyay@2081        [0.68810403, -0.13327494, 0.58295095, 1.208180...
Chiranjivi Keshav@1022      [0.4200372, -1.0032113, 0.9765124, -0.06865919...
Saket Meshram@1066          [1.4751345, 0.5762918, 0.9056971, -0.4449255, ...
Length: 80, dtype: object

In [8]:
retrieve_df=retrive_series.to_frame().reset_index()
retrieve_df.columns=['name_roll','facial features']
retrieve_df[['Name','Roll']]=retrieve_df['name_roll'].apply(lambda x: x.split('@')).apply(pd.Series)
retrieve_df

,name_roll,facial features,Name,Roll
0,Mallavarapu Sai@1044,"[-0.6631383, 0.62165254, -0.9634238, 0.6793268...",Mallavarapu Sai,1044
1,Darshil Patel@1024,"[0.63586926, 1.2339267, 2.0949066, -0.10146476...",Darshil Patel,1024
2,Pradeep Kumar@1058,"[1.2769926, 0.40290147, -0.4040916, 1.5141966,...",Pradeep Kumar,1058
3,Kodudula Niketh@1040,"[0.41742858, 0.9092604, 1.1397063, -1.0795174,...",Kodudula Niketh,1040
4,Panchangam Akhilesh@1053,"[1.3760132, -0.30991262, 0.91287786, 1.0763574...",Panchangam Akhilesh,1053
...,...,...,...,...
75,Aditya Mastud@1005,"[-0.4629953, -0.071729705, -0.083244145, -0.42...",Aditya Mastud,1005
76,S Ruthvik@1064,"[0.81026226, -0.01492546, 0.75623745, 1.347576...",S Ruthvik,1064
77,Vedant Upadhyay@2081,"[0.68810403, -0.13327494, 0.58295095, 1.208180...",Vedant Upadhyay,2081
78,Chiranjivi Keshav@1022,"[0.4200372, -1.0032113, 0.9765124, -0.06865919...",Chiranjivi Keshav,1022


In [11]:
cap=cv2.VideoCapture(0)

while(True):
    ret,frame=cap.read()
    if (ret==False):
        break
    pred_frame=face_recognition.face_prediction(frame,retrieve_df,'facial features',['Name','Roll'],thresh=0.5)

    cv2.imshow('frame',frame)
    cv2.imshow('prediction',pred_frame)
    
    if cv2.waitKey(1)==27:
        break
cv2.destroyAllWindows(0)
cap.release()

d:\face_detection_project\attendance_system1\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
d:\face_detection_project\attendance_system1\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
d:\face_detection_project\attendance_system1\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcon

SystemError: <built-in function destroyAllWindows> returned NULL without setting an exception

In [12]:
cap.release()

: 

In [1]:
import redis

In [3]:
#connection to database
#Connect to redis client

# hostname='redis-16971.c267.us-east-1-4.ec2.cloud.redislabs.com'
# portnumber=16971
# password='OCJ0u9Mnpa050AekK1Z1EMgrTcTWoiLe'

# c=redis.StrictRedis(host=hostname,
#                     port=portnumber,
#                     password=password)

In [4]:
# c.ping()

TimeoutError: Timeout connecting to server